In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline

C:\Users\hp\AppData\Roaming\Python\Python39\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_excel('flight_price.xlsx')
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [4]:
df.describe()

,Price
count,10683.000000
mean,9087.064121
std,4611.359167
min,1759.000000
25%,5277.000000
50%,8372.000000
75%,12373.000000
max,79512.000000


In [5]:
df.isna().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

In [6]:
df[df['Total_Stops'].isna()]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
9039,Air India,6/05/2019,Delhi,Cochin,NaN,09:45,09:25 07 May,23h 40m,NaN,No info,7480


In [7]:
df.head(2)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662


In [8]:
df['Date_Journey']=df['Date_of_Journey'].str.split('/').str[0].astype(int)
df['Month_Journey']=df['Date_of_Journey'].str.split('/').str[1].astype(int)
df['Year_Journey']=df['Date_of_Journey'].str.split('/').str[2].astype(int)
df.drop('Date_of_Journey', axis=1, inplace=True)

In [9]:
df.head(2)

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Date_Journey,Month_Journey,Year_Journey
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897,24,3,2019
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662,1,5,2019


In [10]:
df['Dep_Time_Hr']=df['Dep_Time'].str.split(':').str[0].astype(int)
df['Dep_Time_min']=df['Dep_Time'].str.split(':').str[1].astype(int)
df.drop('Dep_Time', axis=1, inplace=True)
df.head(2)

,Airline,Source,Destination,Route,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Date_Journey,Month_Journey,Year_Journey,Dep_Time_Hr,Dep_Time_min
0,IndiGo,Banglore,New Delhi,BLR → DEL,01:10 22 Mar,2h 50m,non-stop,No info,3897,24,3,2019,22,20
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,13:15,7h 25m,2 stops,No info,7662,1,5,2019,5,50


In [11]:
df['Arrival_Time'].str.split(' ').str[0].str.split(':')

0        [01, 10]
1        [13, 15]
2        [04, 25]
3        [23, 30]
4        [21, 35]
           ...   
10678    [22, 25]
10679    [23, 20]
10680    [11, 20]
10681    [14, 10]
10682    [19, 15]
Name: Arrival_Time, Length: 10683, dtype: object

In [12]:
df['Arrival_Time_Hr']=df['Arrival_Time'].str.split(' ').str[0].str.split(':').str[0].astype(int)
df['Arrival_Time_min']=df['Arrival_Time'].str.split(' ').str[0].str.split(':').str[1].astype(int)
df.drop('Arrival_Time', axis=1, inplace=True)
df.head(2)

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Date_Journey,Month_Journey,Year_Journey,Dep_Time_Hr,Dep_Time_min,Arrival_Time_Hr,Arrival_Time_min
0,IndiGo,Banglore,New Delhi,BLR → DEL,2h 50m,non-stop,No info,3897,24,3,2019,22,20,1,10
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,7h 25m,2 stops,No info,7662,1,5,2019,5,50,13,15


In [13]:
df['Duration'].str.split(' ').str[1].str.split('m').str[0].fillna(30).astype(int)

0        50
1        25
2        30
3        25
4        45
         ..
10678    30
10679    35
10680    30
10681    40
10682    20
Name: Duration, Length: 10683, dtype: int32

In [14]:
df['Duration_Hr']=df['Duration'].str.split(' ').str[0].str.split('h').str[0].str.split('m').str[0].astype(int)
df['Duration_min']=df['Duration'].str.split(' ').str[1].str.split('m').str[0].fillna(30).astype(int)
df.drop('Duration', axis=1, inplace=True)
df.head(2)

,Airline,Source,Destination,Route,Total_Stops,Additional_Info,Price,Date_Journey,Month_Journey,Year_Journey,Dep_Time_Hr,Dep_Time_min,Arrival_Time_Hr,Arrival_Time_min,Duration_Hr,Duration_min
0,IndiGo,Banglore,New Delhi,BLR → DEL,non-stop,No info,3897,24,3,2019,22,20,1,10,2,50
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,2 stops,No info,7662,1,5,2019,5,50,13,15,7,25


In [15]:
df['Total_Stops'].unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [16]:
df['Total_Stops'].isna().sum()

1

In [17]:
df['Total_Stops'][df['Total_Stops'].isna()]

9039    NaN
Name: Total_Stops, dtype: object

In [18]:
df['Total_Stops'].mode()

0    1 stop
Name: Total_Stops, dtype: object

In [19]:
df['Total_Stops']=df['Total_Stops'].map({'non-stop':0, '2 stops':2, '1 stop':1, '3 stops':3, np.nan:1, '4 stops':4})

In [20]:
df.head(2)

,Airline,Source,Destination,Route,Total_Stops,Additional_Info,Price,Date_Journey,Month_Journey,Year_Journey,Dep_Time_Hr,Dep_Time_min,Arrival_Time_Hr,Arrival_Time_min,Duration_Hr,Duration_min
0,IndiGo,Banglore,New Delhi,BLR → DEL,0,No info,3897,24,3,2019,22,20,1,10,2,50
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,2,No info,7662,1,5,2019,5,50,13,15,7,25


In [21]:
#Note : Route has same information as of source and destination column. so we drop Route column

In [22]:
df.drop('Route', axis=1, inplace= True)

In [23]:
df.head(2)

,Airline,Source,Destination,Total_Stops,Additional_Info,Price,Date_Journey,Month_Journey,Year_Journey,Dep_Time_Hr,Dep_Time_min,Arrival_Time_Hr,Arrival_Time_min,Duration_Hr,Duration_min
0,IndiGo,Banglore,New Delhi,0,No info,3897,24,3,2019,22,20,1,10,2,50
1,Air India,Kolkata,Banglore,2,No info,7662,1,5,2019,5,50,13,15,7,25


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Airline           10683 non-null  object
 1   Source            10683 non-null  object
 2   Destination       10683 non-null  object
 3   Total_Stops       10683 non-null  int64 
 4   Additional_Info   10683 non-null  object
 5   Price             10683 non-null  int64 
 6   Date_Journey      10683 non-null  int32 
 7   Month_Journey     10683 non-null  int32 
 8   Year_Journey      10683 non-null  int32 
 9   Dep_Time_Hr       10683 non-null  int32 
 10  Dep_Time_min      10683 non-null  int32 
 11  Arrival_Time_Hr   10683 non-null  int32 
 12  Arrival_Time_min  10683 non-null  int32 
 13  Duration_Hr       10683 non-null  int32 
 14  Duration_min      10683 non-null  int32 
dtypes: int32(9), int64(2), object(4)
memory usage: 876.5+ KB


In [25]:
## we have now 4 columns as object type i.e categorical features. we can use one hot encoding methodZ

In [26]:
df['Airline'].unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [27]:
df['Source'].unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

In [28]:
df['Destination'].unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

In [29]:
df['Additional_Info'].unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

In [30]:
encoder = OneHotEncoder()

In [31]:
df1= pd.DataFrame(encoder.fit_transform(df[['Airline', 'Source','Destination']]).toarray(), columns=encoder.get_feature_names_out())

In [32]:
df1.head(10)

,Airline_Air Asia,Airline_Air India,Airline_GoAir,Airline_IndiGo,Airline_Jet Airways,Airline_Jet Airways Business,Airline_Multiple carriers,Airline_Multiple carriers Premium economy,Airline_SpiceJet,Airline_Trujet,...,Source_Chennai,Source_Delhi,Source_Kolkata,Source_Mumbai,Destination_Banglore,Destination_Cochin,Destination_Delhi,Destination_Hyderabad,Destination_Kolkata,Destination_New Delhi
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [33]:
### drop OneHotEncoded columns and Additional_Info columns and then merge 2 Data Frames

In [34]:
df.drop(['Airline', 'Source','Destination', 'Additional_Info'], axis=1,inplace=True )

In [35]:
df2= pd.DataFrame(df["Price"])
df.drop(['Price'], axis=1, inplace=True)

In [36]:
df.head(2)

,Total_Stops,Date_Journey,Month_Journey,Year_Journey,Dep_Time_Hr,Dep_Time_min,Arrival_Time_Hr,Arrival_Time_min,Duration_Hr,Duration_min
0,0,24,3,2019,22,20,1,10,2,50
1,2,1,5,2019,5,50,13,15,7,25


In [38]:
# Concatenate DataFrames along columns
final_df = pd.concat([df, df1, df2], axis=1)




In [39]:
final_df.head()

,Total_Stops,Date_Journey,Month_Journey,Year_Journey,Dep_Time_Hr,Dep_Time_min,Arrival_Time_Hr,Arrival_Time_min,Duration_Hr,Duration_min,...,Source_Delhi,Source_Kolkata,Source_Mumbai,Destination_Banglore,Destination_Cochin,Destination_Delhi,Destination_Hyderabad,Destination_Kolkata,Destination_New Delhi,Price
0,0,24,3,2019,22,20,1,10,2,50,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3897
1,2,1,5,2019,5,50,13,15,7,25,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,7662
2,2,9,6,2019,9,25,4,25,19,30,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,13882
3,1,12,5,2019,18,5,23,30,5,25,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6218
4,1,1,3,2019,16,50,21,35,4,45,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,13302


In [40]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 34 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Total_Stops                                10683 non-null  int64  
 1   Date_Journey                               10683 non-null  int32  
 2   Month_Journey                              10683 non-null  int32  
 3   Year_Journey                               10683 non-null  int32  
 4   Dep_Time_Hr                                10683 non-null  int32  
 5   Dep_Time_min                               10683 non-null  int32  
 6   Arrival_Time_Hr                            10683 non-null  int32  
 7   Arrival_Time_min                           10683 non-null  int32  
 8   Duration_Hr                                10683 non-null  int32  
 9   Duration_min                               10683 non-null  int32  
 10  Airline_Air Asia      

In [41]:
final_df.describe()

,Total_Stops,Date_Journey,Month_Journey,Year_Journey,Dep_Time_Hr,Dep_Time_min,Arrival_Time_Hr,Arrival_Time_min,Duration_Hr,Duration_min,...,Source_Delhi,Source_Kolkata,Source_Mumbai,Destination_Banglore,Destination_Cochin,Destination_Delhi,Destination_Hyderabad,Destination_Kolkata,Destination_New Delhi,Price
count,10683.000000,10683.000000,10683.000000,10683.0,10683.000000,10683.000000,10683.000000,10683.000000,10683.000000,10683.000000,...,10683.000000,10683.000000,10683.000000,10683.000000,10683.000000,10683.000000,10683.000000,10683.000000,10683.000000,10683.000000
mean,0.824207,13.508378,4.708602,2019.0,12.490686,24.411214,13.348778,24.690630,10.246560,31.225311,...,0.424693,0.268745,0.065244,0.268745,0.424693,0.118412,0.065244,0.035664,0.087241,9087.064121
std,0.675199,8.479277,1.164357,0.0,5.748650,18.767980,6.859125,16.506036,8.494988,14.195688,...,0.494320,0.443328,0.246967,0.443328,0.494320,0.323111,0.246967,0.185460,0.282202,4611.359167
min,0.000000,1.000000,3.000000,2019.0,0.000000,0.000000,0.000000,0.000000,1.000000,5.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1759.000000
25%,0.000000,6.000000,3.000000,2019.0,8.000000,5.000000,8.000000,10.000000,2.000000,20.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5277.000000
50%,1.000000,12.000000,5.000000,2019.0,11.000000,25.000000,14.000000,25.000000,8.000000,30.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8372.000000
75%,1.000000,21.000000,6.000000,2019.0,18.000000,40.000000,19.000000,35.000000,15.000000,45.000000,...,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,12373.000000
max,4.000000,27.000000,6.000000,2019.0,23.000000,55.000000,23.000000,55.000000,47.000000,55.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,79512.000000
